In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sma
from scipy.stats.mstats import winsorize
from scipy.stats import gmean
import scipy.stats as scs
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
# read data
DF_sm = pd.read_csv('company_stock_1999-2021.csv', parse_dates=['datadate'])
print('read company stock data')

DF_fd = pd.read_csv('financial_data_1999-2021.csv', parse_dates=['datadate'])
print('read financial data')

DF_snpd=pd.read_csv('snp_daily.csv',parse_dates=['Date'])
print('read snp data')

'Set up files to write output'

output_file = open('output.csv', 'w')

read company stock data
read financial data
read snp data


In [3]:
'Calculate and assign risk free rate Y1999 - Y2020'

#Data Source of US treasury rates -> https://fred.stlouisfed.org/series/DGS10
rfdf = pd.read_csv('DGS10.csv')

#Remove all '.' information from non-trading days
rfdf = rfdf[rfdf.DGS10 != '.']

rfdf['Year'] = rfdf['DATE'].str[:4] 

#Convert all data in 'DGS10' column to numeric data
rfdf = rfdf[['DGS10']].apply(pd.to_numeric)

# Assign Average Risk Free Rate rf
rf = rfdf['DGS10'].mean() / 100
rf

print('risk free rf created')

risk free rf created


In [4]:
'SORTING VALUES AND DUPLICATE DATAFRAMES'

#Sort Values (gvkey and date)
DF_sm = DF_sm.sort_values(by=['gvkey','datadate'])
DF_sm = DF_sm.dropna()
# Using only stock iid= 01
DF_sm = DF_sm[DF_sm['iid'] == '01']     

DF_fd = DF_fd.sort_values(by=['gvkey','datadate'])
DF_fd = DF_fd[DF_fd['mkvalt'] > 100]
DF_fd = DF_fd[DF_fd['gsector'] != 40] 

#Duplicate Dataframes
df_sm = DF_sm
df_fd = DF_fd
df_snpd=DF_snpd

In [5]:
'Create portfolio formation year (pfy)'

df_sm['year'], df_sm['month'] = df_sm['datadate'].dt.year, df_sm['datadate'].dt.month
df_sm['pfy'] = np.where(df_sm.month > 6, df_sm.year, df_sm.year - 1) 
print('Get pfy of company sotck data')


df_fd['year'], df_fd['month'] = df_fd['datadate'].dt.year, df_fd['datadate'].dt.month
df_fd['pfy'] = np.where(df_fd.month < 4 , df_fd.year, df_fd.year + 1)
print('Get pfy of financial data')


df_snpd['year'], df_snpd['month'] = df_snpd['Date'].dt.year, df_snpd['Date'].dt.month
df_snp=df_snpd.groupby(['year','month']).tail(1)
df_snp['pfy'] = np.where(df_snp.month > 6 , df_snp.year, df_snp.year - 1)
print('Get pfy of S&P')

Get pfy of company sotck data
Get pfy of financial data
Get pfy of S&P


In [6]:
'Calculate the S&P return for each pfy'
df_snp['snpret'] = df_snp.sort_values(by='Date')['Adj Close'].pct_change()
df_snp['snpret1'] = 1 + df_snp['snpret'] 

snp1=df_snp[df_snp.pfy !=1998]
snp1=snp1[snp1.pfy != 2021]
snp1['snpyret']=snp1.groupby(['pfy'])['snpret1'].cumprod() -1
snpy=snp1.groupby(['pfy']).nth(11)
snpy=snpy[['snpyret']]
print('Calculate the annual return of the S&P 500')

Calculate the annual return of the S&P 500


In [7]:
'Calculate the annual return for each company based on gvkey and pfy'

df_sm['csmret1'] = 1 + df_sm.trt1m/100
df_sm['csyret'] = df_sm.groupby(['gvkey', 'pfy'])['csmret1'].cumprod() - 1
df_sm1=df_sm.groupby(['gvkey','pfy']).nth(11)
wl=0.025
df_sm1['csyret']=winsorize(df_sm1['csyret'],limits=[wl,wl])
df_csyret=df_sm1[['csyret']]
print('Calculated pfy returns of company sotck')

Calculated pfy returns of company sotck


In [8]:
'Simple Moving Averages Creation'

#            6, 12, 18 months moving averages
df_sm['6_M'] = df_sm.groupby('gvkey')['prccm'].transform(lambda prccm: prccm.rolling(6, center=False).mean()).shift()
df_sm['12_M'] = df_sm.groupby('gvkey')['prccm'].transform(lambda prccm: prccm.rolling(12, center=False).mean()).shift()
df_sm['18_M'] = df_sm.groupby('gvkey')['prccm'].transform(lambda prccm: prccm.rolling(18, center=False).mean()).shift()

df_Momentum=df_sm[['gvkey','pfy','6_M','12_M','18_M','month','year','prccm']].dropna()
df_Momentum=df_Momentum[df_Momentum['month']==6]

df_Momentum['6_MF'] = np.where(df_Momentum['6_M'] > df_Momentum.prccm, 0, 1)
df_Momentum['12_MF'] = np.where(df_Momentum['12_M'] > df_Momentum.prccm, 0, 1)
df_Momentum['18_MF'] = np.where(df_Momentum['18_M'] > df_Momentum.prccm, 0, 1)
df_Momentum=df_Momentum[['gvkey','pfy','6_MF','12_MF','18_MF']]
print('get monmentum factor')

get monmentum factor


In [9]:
'Piotroski F-score'

###-Profitability
#Return on Asset = Income before extraordinary items / Total Assets
df_fd['roa'] = df_fd['act'] - df_fd['at']

#Operating Cash Flow = Operating Activities Net Cash Flow / Total Assets 
df_fd['cfo'] = df_fd['oancf'] / df_fd['at']

#Lagged Return of Assets
df_fd['roa_t-1'] = df_fd.sort_values(['pfy']).groupby('gvkey')['roa'].shift(1)

#Accurals = (Income Before Extraordinary Items - Operating Activities Net Cash Flow) / Total Assets
df_fd['accurals'] = (df_fd['ib'] - df_fd['oancf']) / df_fd['at']


###-Leverage, Liquidity, and Source of Funds
#Leverage = Total Long-Term Debt / Total Assets
df_fd['leverage'] = df_fd['dltt'] / df_fd['at']
#Current Ratio = Total Current Assets / Total Current Liabilities
df_fd['curr_ratio'] = df_fd['act'] / df_fd['lct']

#Lagged Leverage
df_fd['leverage_t-1'] = df_fd.sort_values(['pfy']).groupby('gvkey')['leverage'].shift(1)
#Lagged Current Ratio
df_fd['curr_ratio_t-1'] = df_fd.sort_values(['pfy']).groupby('gvkey')['curr_ratio'].shift(1)
#Lagged Ordinary Share
df_fd['ceq_t-1'] = df_fd.sort_values(['pfy']).groupby('gvkey')['ceq'].shift(1)

###-Operating Efficiency
#Gross Margin = (Sales/Turnover (Net) - COGS) / Sales (Net)
df_fd['margin'] = (df_fd['sale'] - df_fd['cogs']) / df_fd['sale'] 
#Asset Turover ratio = Sales/Turnover (Net) / Total Assets
df_fd['turn'] = df_fd['sale'] / df_fd['at']

#Lagged Gross Margin
df_fd['margin_t-1'] = df_fd.sort_values(['pfy']).groupby('gvkey')['margin'].shift(1)
#Lagged Asset Turnover ratio
df_fd['turn_t-1'] = df_fd.sort_values(['pfy']).groupby('gvkey')['turn'].shift(1)

def fscore(row):
    fscore = 0
    if row['roa'] > 0:
        fscore += 1
    if row['cfo'] > 0:
        fscore += 1
    if row['roa'] > row['roa_t-1']:
        fscore += 1
    if row['cfo'] > row['roa']:
        fscore += 1
    if row['leverage'] < row['leverage_t-1']:
        fscore += 1  
    if row['curr_ratio'] > row['curr_ratio_t-1']:
        fscore += 1
    if row['ceq'] == row['ceq_t-1']:
        fscore += 1
    if row['margin'] > row['margin_t-1']:
        fscore += 1
    if row['turn'] > row['turn_t-1']:
        fscore += 1
    return fscore

df_fd['Fscore'] = df_fd.apply(fscore, axis=1)

#For use in later calculation of other accounting variables for models

df_Fscore = df_fd[['gvkey', 'pfy', 'Fscore']]

print('Get F-score')

Get F-score


In [10]:
'Altman Z-score'
# Create Z-Score
df_fd['Zscore'] = (df_fd['ebit'] / df_fd['at']) * 3.3 + (df_fd['sale'] / df_fd['at']) * 1.0 + (df_fd['ceq'] / df_fd['at']) * 0.6 + (df_fd['wcap'] / df_fd['at']) * 1.2 + (df_fd['re'] / df_fd['at']) * 1.4

df_Zscore = df_fd[['gvkey', 'pfy', 'Zscore']]

print('Get Z-score')

Get Z-score


In [11]:
'Fama & French - Small minus big (SMB)'

df_fd['logmv']=np.log(df_fd['mkvalt'])
df_lgmv = df_fd[['gvkey', 'pfy', 'logmv']]
df_lgmv['logmv'] = winsorize(df_lgmv['logmv'], limits=[wl, wl])
df_lgmv['d_logmkvalt'] = df_lgmv.groupby('pfy')['logmv'].transform(pd.qcut, 10, labels=False)

print('Get logmv')

Get logmv


In [12]:
'Fama & French - High minus low (HML)'

df_fd['bm'] = df_fd['ceq'] / df_fd['mkvalt']
df_bm = df_fd[['gvkey', 'pfy', 'bm']]
df_bm['bm'] = winsorize(df_bm['bm'], limits=[wl, wl])
df_bm['bm_dec'] = df_bm.groupby('pfy')['bm'].transform(pd.qcut, 10, labels=False)

print('Get BM ratio')

Get BM ratio


In [13]:
'Other Accounting Variables to use for models'


df_fd['e/m'] = df_fd['ebit'] / df_fd['mkvalt']
df_fd['asset_turn'] = df_fd['revt'] / df_fd['at']

df_acc= df_fd[['gvkey', 'pfy', 'e/m', 'asset_turn']].replace([np.inf, -np.inf], np.nan).dropna()

df_acc['e/m'] = winsorize(df_acc['e/m'], limits=[wl, wl])
df_acc['asset_turn'] = winsorize(df_acc['asset_turn'], limits=[wl, wl])

In [14]:
'Merge Dataframes'
df_merged2 = pd.merge(df_csyret,df_Fscore, how='inner', on=['gvkey', 'pfy'])
df_merged3 = pd.merge(df_merged2,df_Zscore, how='inner', on=['gvkey', 'pfy'])
df_merged4 = pd.merge(df_merged3,df_acc, how='inner', on=['gvkey', 'pfy'])
df_merged5 = pd.merge(df_merged4,df_lgmv, how='inner', on=['gvkey', 'pfy'])
df_merged6 = pd.merge(df_merged5,df_bm, how='inner', on=['gvkey', 'pfy'])
df_merged7 = pd.merge(df_merged6,df_Momentum, how='inner', on=['gvkey', 'pfy'])
df_complete = pd.merge(df_merged7,snpy, how='inner', on=['pfy']).dropna()
print('Merge all data')

Merge all data


In [15]:
'Output and Save df_final.csv'

df_complete.to_csv('df_omplete.csv')
print('output final df','\n'*5)

output final df 







In [16]:
df_complete.groupby('bm_dec')[['bm','csyret']].agg(['count', 'mean'])

bm           csyret          
       count      mean  count      mean
bm_dec                                 
0       3152  0.001775   3152  0.089766
1       3412  0.133927   3412  0.119979
2       3417  0.213523   3417  0.122732
3       3347  0.284259   3347  0.135158
4       3270  0.354044   3270  0.133378
5       3219  0.431166   3219  0.135556
6       3110  0.523551   3110  0.144978
7       3122  0.637777   3122  0.131631
8       2927  0.807195   2927  0.150883
9       2930  1.223798   2930  0.150115

In [17]:
print(scs.ttest_ind(df_complete['csyret'],df_complete['Fscore'],equal_var=False))

Ttest_indResult(statistic=-429.54194520193846, pvalue=0.0)


In [18]:
'Model 1 - OLS Model w/o momemtum'
df_model1 = df_complete

X = df_model1[['e/m', 'asset_turn','logmv','bm']]

X = sma.add_constant(X)
y =  df_model1['csyret']

model1 = sma.OLS(y,X)
results = model1.fit()
print('\n', 'Model 1 - OLS Model w/o momemtum')
print(results.summary())
print()
print('\n'*5)


 Model 1 - OLS Model w/o momemtum
                            OLS Regression Results                            
Dep. Variable:                 csyret   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     33.55
Date:                Thu, 16 Jun 2022   Prob (F-statistic):           5.69e-28
Time:                        20:21:23   Log-Likelihood:                -26607.
No. Observations:               31906   AIC:                         5.322e+04
Df Residuals:                   31901   BIC:                         5.327e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.

In [19]:
'Model 2 - OLS Model with 6 month momemtum'
df_model2 = df_complete

X = df_model2[['e/m', 'asset_turn', 'logmv','bm','6_MF']]

X = sma.add_constant(X)
y =  df_model2['csyret']

model2 = sma.OLS(y,X)
results = model2.fit()
print('\n', 'Model 2 - OLS Model with 6 month momemtum')
print(results.summary())
print()
print('\n'*5)


 Model 2 - OLS Model with 6 month momemtum
                            OLS Regression Results                            
Dep. Variable:                 csyret   R-squared:                       0.135
Model:                            OLS   Adj. R-squared:                  0.135
Method:                 Least Squares   F-statistic:                     992.9
Date:                Thu, 16 Jun 2022   Prob (F-statistic):               0.00
Time:                        20:21:23   Log-Likelihood:                -24366.
No. Observations:               31906   AIC:                         4.874e+04
Df Residuals:                   31900   BIC:                         4.879e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const   

In [20]:
'Model 3 - OLS Model with 12 month momemtum'
df_model3 = df_complete

X = df_model3[['e/m', 'asset_turn', 'logmv','bm','12_MF']]

X = sma.add_constant(X)
y =  df_model3['csyret']

model3 = sma.OLS(y,X)
results = model3.fit()
print('\n', 'Model 3 - OLS Model with 12 month momemtum')
print(results.summary())
print()
print('\n'*5)


 Model 3 - OLS Model with 12 month momemtum
                            OLS Regression Results                            
Dep. Variable:                 csyret   R-squared:                       0.301
Model:                            OLS   Adj. R-squared:                  0.301
Method:                 Least Squares   F-statistic:                     2751.
Date:                Thu, 16 Jun 2022   Prob (F-statistic):               0.00
Time:                        20:21:23   Log-Likelihood:                -20954.
No. Observations:               31906   AIC:                         4.192e+04
Df Residuals:                   31900   BIC:                         4.197e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const  

In [21]:
'Model 4 - OLS Model with 18 month momemtum'
df_model4 = df_complete

X = df_model4[['e/m','asset_turn', 'logmv','bm','18_MF']]

X = sma.add_constant(X)
y =  df_model4['csyret']

model4 = sma.OLS(y,X)
results = model4.fit()
print('\n', 'Model 4 - OLS Model with 18 month momemtum')
print(results.summary())
print()
print('\n'*5)


 Model 4 - OLS Model with 18 month momemtum
                            OLS Regression Results                            
Dep. Variable:                 csyret   R-squared:                       0.350
Model:                            OLS   Adj. R-squared:                  0.350
Method:                 Least Squares   F-statistic:                     3431.
Date:                Thu, 16 Jun 2022   Prob (F-statistic):               0.00
Time:                        20:21:23   Log-Likelihood:                -19808.
No. Observations:               31906   AIC:                         3.963e+04
Df Residuals:                   31900   BIC:                         3.968e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const  

In [22]:
'Model 5 - OLS Model with 6 month momemtum except e/m'
df_model5 = df_complete

X = df_model5[['asset_turn', 'logmv','bm','6_MF']]

X = sma.add_constant(X)
y =  df_model5['csyret']

model5 = sma.OLS(y,X)
results = model5.fit()
print('\n', 'Model 5 - OLS Model with 6 month momemtum except e/m')
print(results.summary())
print()
print('\n'*5)


 Model 5 - OLS Model with 6 month momemtum except e/m
                            OLS Regression Results                            
Dep. Variable:                 csyret   R-squared:                       0.135
Model:                            OLS   Adj. R-squared:                  0.135
Method:                 Least Squares   F-statistic:                     1241.
Date:                Thu, 16 Jun 2022   Prob (F-statistic):               0.00
Time:                        20:21:23   Log-Likelihood:                -24367.
No. Observations:               31906   AIC:                         4.874e+04
Df Residuals:                   31901   BIC:                         4.879e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------

In [23]:
'Model 6 - OLS Model with 12 month momemtum except e/m'
df_model6 = df_complete

X = df_model6[['asset_turn', 'logmv','bm','12_MF']]

X = sma.add_constant(X)
y =  df_model6['csyret']

model6 = sma.OLS(y,X)
results = model6.fit()
print('\n', 'Model 6 - OLS Model with 12 month momemtum except e/m')
print(results.summary())
print()
print('\n'*5)


 Model 6 - OLS Model with 12 month momemtum except e/m
                            OLS Regression Results                            
Dep. Variable:                 csyret   R-squared:                       0.301
Model:                            OLS   Adj. R-squared:                  0.301
Method:                 Least Squares   F-statistic:                     3439.
Date:                Thu, 16 Jun 2022   Prob (F-statistic):               0.00
Time:                        20:21:24   Log-Likelihood:                -20954.
No. Observations:               31906   AIC:                         4.192e+04
Df Residuals:                   31901   BIC:                         4.196e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------

In [24]:
'Model 7 - OLS Model with 18 month momemtum except e/m'
df_model7 = df_complete

X = df_model7[['asset_turn', 'logmv','bm','18_MF']]

X = sma.add_constant(X)
y =  df_model7['csyret']

model7 = sma.OLS(y,X)
results = model7.fit()
print('\n', 'Model 7 - OLS Model with 18 month momemtum except e/m')
print(results.summary())
print()
print('\n'*5)


 Model 7 - OLS Model with 18 month momemtum except e/m
                            OLS Regression Results                            
Dep. Variable:                 csyret   R-squared:                       0.350
Model:                            OLS   Adj. R-squared:                  0.350
Method:                 Least Squares   F-statistic:                     4287.
Date:                Thu, 16 Jun 2022   Prob (F-statistic):               0.00
Time:                        20:21:24   Log-Likelihood:                -19811.
No. Observations:               31906   AIC:                         3.963e+04
Df Residuals:                   31901   BIC:                         3.967e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------

In [25]:
df_complete.describe()

,gvkey,pfy,csyret,Fscore,Zscore,e/m,asset_turn,logmv,d_logmkvalt,bm,bm_dec,6_MF,12_MF,18_MF,snpyret
count,31906.000000,31906.000000,31906.000000,31906.000000,31906.000000,31906.000000,31906.000000,31906.000000,31906.000000,31906.000000,31906.000000,31906.000000,31906.000000,31906.000000,31906.000000
mean,58959.024886,2011.460008,0.131006,3.875979,-1.377603,0.058853,0.931417,7.244703,4.497681,0.446844,4.380430,0.543189,0.551244,0.543001,0.080267
std,65139.571303,5.960696,0.558272,1.453821,211.151873,0.102558,0.684802,1.647600,2.902794,0.350237,2.840239,0.498139,0.497375,0.498155,0.155943
min,1004.000000,2000.000000,-0.946002,0.000000,-27942.650000,-0.262662,0.000000,4.754078,0.000000,-0.100880,0.000000,0.000000,0.000000,0.000000,-0.281781
25%,9225.000000,2007.000000,-0.185578,3.000000,0.789557,0.020762,0.421904,5.905496,2.000000,0.202793,2.000000,0.000000,0.000000,0.000000,0.031439
50%,25747.000000,2012.000000,0.074683,4.000000,1.835650,0.069161,0.785117,7.065560,4.000000,0.373514,4.000000,1.000000,1.000000,1.000000,0.082178
75%,116504.000000,2017.000000,0.343509,5.000000,2.797736,0.109399,1.284006,8.360644,7.000000,0.610121,7.000000,1.000000,1.000000,1.000000,0.170693
max,331856.000000,2020.000000,4.000000,7.000000,33.573675,0.308402,2.847419,11.024109,9.000000,1.585157,9.000000,1.000000,1.000000,1.000000,0.386161


In [26]:
#Portfolio Creation
#Fscore > 4 
#Zscore > 2.0 --> only choose stocks of firms that is well above 1.8 threshold value that indicates if firm is heeding for bankruptcy
#bm_dec > 4 --> only choose high book to market to earn value stocks
#asset_turn > 0.785
#d_logmkvalt < 6

#Portfolio 0 - no SMA
df_pf_no_M = df_complete.loc[(df_complete.Fscore > 4) & (df_complete.bm_dec > 4) & (df_complete.d_logmkvalt < 6) & (df_complete.Zscore > 2.0) & (df_complete.asset_turn >0.785)]

#Portfolio 1 - 6 month SMA
df_pf_6_M = df_complete.loc[ (df_complete.Fscore > 4) & (df_complete.bm_dec > 4) & (df_complete.d_logmkvalt < 6) & (df_complete.Zscore > 2.0)& (df_complete.asset_turn >0.785) & (df_complete['6_MF'] == 1) ]

#Portfolio 2 - 12 month SMA
df_pf_12_M = df_complete.loc[ (df_complete.Fscore > 4) & (df_complete.bm_dec > 4) & (df_complete.d_logmkvalt < 6) & (df_complete.Zscore > 2.0)& (df_complete.asset_turn >0.785) & (df_complete['12_MF'] == 1) ]

#Portfolio 3 - 18 month SMA
df_pf_18_M = df_complete.loc[ (df_complete.Fscore > 4) & (df_complete.bm_dec > 4) & (df_complete.d_logmkvalt < 6) & (df_complete.Zscore > 2.0)& (df_complete.asset_turn >0.785) & (df_complete['18_MF'] == 1) ]

print('created portfolios')

created portfolios


In [27]:
'Returns Calculations'
df_pfnoM = df_pf_no_M.groupby(['pfy'])['csyret'].mean()
df_pf6M = df_pf_6_M.groupby(['pfy'])['csyret'].mean()
df_pf12M = df_pf_12_M.groupby(['pfy'])['csyret'].mean()
df_pf18M = df_pf_18_M.groupby(['pfy'])['csyret'].mean()

df_mer1 = pd.merge(df_pfnoM,df_pf6M, how='inner', on=['pfy'])
df_mer2 = pd.merge(df_mer1,df_pf12M, how='inner', on=['pfy'])
df_mer3 = pd.merge(df_mer2,df_pf18M, how='inner', on=['pfy'])
df_pf_return = pd.merge(df_mer3,snpy, how='inner', on=['pfy'])
df_pf_return.columns=['pfnoM_return','pf6M_return','pf12M_return','pf18M_return','snpyret']
print('Calculated annual returns on universe and portfolio')
df_pf_return

Calculated annual returns on universe and portfolio


,pfnoM_return,pf6M_return,pf12M_return,pf18M_return,snpyret
pfy,,,,,
2000,0.470215,0.538788,0.506781,0.538788,-0.158270
2001,0.213927,0.322879,0.389378,0.365186,-0.191575
2002,-0.093487,-0.063695,0.016942,0.069659,-0.015478
2003,0.567845,0.672552,0.678494,0.645711,0.170693
2004,0.168518,0.335259,0.354228,0.371210,0.044257
2005,0.316376,0.474849,0.466775,0.518833,0.066203
2006,0.244525,0.366326,0.393510,0.395678,0.183554
2007,-0.275926,-0.032157,0.168605,0.339412,-0.148568
2008,-0.280437,-0.234241,0.010449,0.175579,-0.281781


In [28]:
#Arithmetic mean return and Geometric mean returns
AMR=df_pf_return.mean()
GMR=gmean(df_pf_return+1,axis=0)-1
GMR = pd.Series(GMR.tolist())
GMR.index=['pfnoM_return','pf6M_return','pf12M_return','pf18M_return','snpyret']
M=pd.concat([AMR,GMR],axis = 1)
M.columns=['Arithmetic mean return','Geometric mean returns']
print('Arithmetic mean return and Geometric mean returns')
M

Arithmetic mean return and Geometric mean returns


,Arithmetic mean return,Geometric mean returns
pfnoM_return,0.162961,0.132293
pf6M_return,0.317289,0.290147
pf12M_return,0.362342,0.348215
pf18M_return,0.380542,0.371159
snpyret,0.065280,0.052940


In [29]:
'Calculate Alpha Returns'
#Portfolio 0 - no SMA
#Based on S&P 500
df_pf_return['Alpha1_pfnoM'] = df_pf_return['pfnoM_return'] - df_pf_return['snpyret']


#Portfolio 1 - 6 M
#Based on  S&P 500
df_pf_return['Alpha1_pf6M'] = df_pf_return['pf6M_return'] - df_pf_return['snpyret']


#Portfolio 2 - 12 M
#Based on  S&P 500
df_pf_return['Alpha1_pf12M'] = df_pf_return['pf12M_return'] - df_pf_return['snpyret']

#Portfolio 3 - 18 M
#Based on  S&P 500
df_pf_return['Alpha1_pf18M'] = df_pf_return['pf18M_return'] - df_pf_return['snpyret']

print('\nCalculated alphas\n\n')
print(df_pf_return)


Calculated alphas


      pfnoM_return  pf6M_return  pf12M_return  pf18M_return   snpyret  \
pfy                                                                     
2000      0.470215     0.538788      0.506781      0.538788 -0.158270   
2001      0.213927     0.322879      0.389378      0.365186 -0.191575   
2002     -0.093487    -0.063695      0.016942      0.069659 -0.015478   
2003      0.567845     0.672552      0.678494      0.645711  0.170693   
2004      0.168518     0.335259      0.354228      0.371210  0.044257   
2005      0.316376     0.474849      0.466775      0.518833  0.066203   
2006      0.244525     0.366326      0.393510      0.395678  0.183554   
2007     -0.275926    -0.032157      0.168605      0.339412 -0.148568   
2008     -0.280437    -0.234241      0.010449      0.175579 -0.281781   
2009      0.320465     0.530742      0.560445      0.467461  0.121166   
2010      0.421082     0.668212      0.609352      0.561721  0.281292   
2011     -0.047088     0.13695

In [30]:
'Sharpe Ratio Calculation'
df_r = df_pf_return[['pfnoM_return', 'pf6M_return', 'pf12M_return', 'pf18M_return']]

Sharpe = (df_r.mean() - rf) / df_r.std()

print('\n','Sharpe ratios:')
print(Sharpe)


 Sharpe ratios:
pfnoM_return    0.483608
pf6M_return     1.072518
pf12M_return    1.654157
pf18M_return    2.098002
dtype: float64


In [31]:
'Sortino Ratio Calculation'
a=df_r.count()
semi_standard_deviation = df_r - df_r.mean()
semi_standard_deviation = semi_standard_deviation.where(semi_standard_deviation < 0,0)
semi_standard_deviation = semi_standard_deviation ** 2
semi_standard_deviation = (semi_standard_deviation.sum()/(a-1)) ** 0.5

Sortino = df_r.mean() / semi_standard_deviation

print('\n','Sortino ratios:')
print(Sortino)


 Sortino ratios:
pfnoM_return    0.849144
pf6M_return     1.634574
pf12M_return    2.546890
pf18M_return    3.391981
dtype: float64


In [32]:
'Maximum Drawdown Calculation'
cumret1=np.cumprod(df_r + 1)
hwm1 = cumret1.cummax()
drawdown1 = ((hwm1 - cumret1) / hwm1) * 100
maxDD_percent = drawdown1.max()
print('\n','Maximum Drawdown:')
print(maxDD_percent)


 Maximum Drawdown:
pfnoM_return    47.898283
pf6M_return     25.886555
pf12M_return     0.000000
pf18M_return     0.000000
dtype: float64


In [33]:
print(scs.ttest_1samp(df_pf_return.Alpha1_pfnoM, 0))
print(scs.ttest_1samp(df_pf_return.Alpha1_pf6M, 0))
print(scs.ttest_1samp(df_pf_return.Alpha1_pf12M, 0))
print(scs.ttest_1samp(df_pf_return.Alpha1_pf18M, 0))

Ttest_1sampResult(statistic=2.110060100723242, pvalue=0.0476481391377326)
Ttest_1sampResult(statistic=5.977686621027403, pvalue=7.607506938252063e-06)
Ttest_1sampResult(statistic=8.69171032889675, pvalue=3.1629383471740916e-08)
Ttest_1sampResult(statistic=9.216685241112588, pvalue=1.224494467797736e-08)
